In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import time

import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
data = pd.read_csv("/content/drive/My Drive/kth/s1p2/ID2223:ScalableML/project/donaldbot/data/tweets-raw.csv")

In [ ]:
data.size

52872

In [ ]:
data.head(10)

,Date,Time,Tweet_Text,Type,Media_Type,Hashtags,Tweet_Id,Tweet_Url,twt_favourites_IS_THIS_LIKE_QUESTION_MARK,Retweets,Unnamed: 10,Unnamed: 11
0,16-11-11,15:26:37,Today we express our deepest gratitude to all ...,text,photo,ThankAVet,7.970000e+17,https://twitter.com/realDonaldTrump/status/797...,127213,41112,NaN,NaN
1,16-11-11,13:33:35,Busy day planned in New York. Will soon be mak...,text,NaN,NaN,7.970000e+17,https://twitter.com/realDonaldTrump/status/797...,141527,28654,NaN,NaN
2,16-11-11,11:14:20,Love the fact that the small groups of protest...,text,NaN,NaN,7.970000e+17,https://twitter.com/realDonaldTrump/status/797...,183729,50039,NaN,NaN
3,16-11-11,2:19:44,Just had a very open and successful presidenti...,text,NaN,NaN,7.970000e+17,https://twitter.com/realDonaldTrump/status/796...,214001,67010,NaN,NaN
4,16-11-11,2:10:46,A fantastic day in D.C. Met with President Oba...,text,NaN,NaN,7.970000e+17,https://twitter.com/realDonaldTrump/status/796...,178499,36688,NaN,NaN
5,16-11-10,19:31:27,Happy 241st birthday to the U.S. Marine Corps!...,text,photo,NaN,7.970000e+17,https://twitter.com/realDonaldTrump/status/796...,159176,44655,NaN,NaN
6,16-11-09,11:36:58,Such a beautiful and important evening! The fo...,text,NaN,NaN,7.960000e+17,https://twitter.com/realDonaldTrump/status/796...,627615,225164,NaN,NaN
7,16-11-09,2:48:27,Watching the returns at 9:45pm.\n#ElectionNigh...,text,photo,ElectionNight;MAGA,7.960000e+17,https://twitter.com/realDonaldTrump/status/796...,185160,45492,NaN,NaN
8,16-11-09,1:35:15,RT @IvankaTrump: Such a surreal moment to vote...,text,NaN,NaN,7.960000e+17,https://twitter.com/realDonaldTrump/status/796...,99809,17169,NaN,NaN
9,16-11-08,23:20:39,RT @EricTrump: Join my family in this incredib...,text,NaN,MakeAmericaGreatAgain;VOTE,7.960000e+17,https://twitter.com/realDonaldTrump/status/796...,63868,19710,NaN,NaN


Data Cleaning

In [ ]:
data = data[pd.isnull(data['Media_Type'])]
data = data[pd.isnull(data['Hashtags'])]

In [ ]:
data = data[data.Type == "text"]
df = data[["Tweet_Text"]]

In [ ]:
df = df[~df.Tweet_Text.str.contains('"')]
df = df[~df.Tweet_Text.str.contains("http")]
df = df[~df.Tweet_Text.str.contains("RT @")]

In [ ]:
df = df.replace({'&amp;': '&'}, regex=True)

In [ ]:
df.to_csv('tweets-cleaned.csv', index=False)

In [ ]:
df

,Tweet_Text
1,Busy day planned in New York. Will soon be mak...
2,Love the fact that the small groups of protest...
3,Just had a very open and successful presidenti...
4,A fantastic day in D.C. Met with President Oba...
6,Such a beautiful and important evening! The fo...
...,...
7366,.@BradSteinle Great talking to you and your pa...
7367,".@andydean2014 Thank you, you were great. You ..."
7369,I hope the boycott of @Macys continues forever...
7370,I loved firing goofball atheist Penn @pennjill...


In [ ]:
from google.colab import files
files.download("tweets-cleaned.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# remove first line  
# only run this one time

# with open("/content/drive/My Drive/kth/s1p2/ID2223:ScalableML/project/donaldbot/data/tweets-cleaned.csv", "r") as fin:
#     data = fin.read().splitlines(True)
# with open("/content/drive/My Drive/kth/s1p2/ID2223:ScalableML/project/donaldbot/data/tweets-cleaned.txt", "w") as fout:
#     fout.writelines(data[1:])

In [ ]:
text = open("/content/drive/My Drive/kth/s1p2/ID2223:ScalableML/project/donaldbot/data/tweets-cleaned.txt", 'rb').read().decode(encoding='utf-8')

print ('Number of Characters: {} characters'.format(len(text)))

Length of text: 293680 characters


Analysis

In [ ]:
# First 250 characters in text
print(text[:100])

Busy day planned in New York. Will soon be making some very important decisions on the people who wi


In [ ]:
# The count of unique characters that are present in the file
vocabulary= sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

85 unique characters


In [ ]:
# To keep track of the number of times each word was used
import operator

word_count= dict()
words = text.split()

for word in words:
    if word in counts:
        counts[word] += 1
    else:
        counts[word] = 1

sorted_1= sorted(counts.items(), key=operator.itemgetter(1),reverse=True)
sorted_2= list(reversed(sorted_counts))

print(sorted_counts)

[('the', 1572), ('to', 1077), ('and', 1008), ('a', 857), ('I', 804), ('in', 794), ('is', 774), ('of', 746), ('on', 701), ('be', 597), ('will', 530), ('for', 519), ('that', 403), ('at', 374), ('-', 296), ('have', 295), ('with', 285), ('are', 280), ('was', 260), ('Hillary', 258), ('by', 252), ('you', 242), ('great', 238), ('my', 231), ('has', 223), ('not', 217), ('&', 206), ('he', 205), ('me', 202), ('The', 201), ('all', 200), ('so', 192), ('very', 178), ('just', 176), ('it', 170), ('his', 164), ('people', 157), ('am', 146), ('who', 145), ('our', 143), ('"I', 141), ('about', 140), ('from', 133), ('out', 131), ('Will', 129), ('Crooked', 129), ('but', 126), ('interviewed', 124), ('they', 123), ('Clinton', 120), ('big', 120), ('her', 117), ('as', 116), ('would', 111), ('like', 108), ('last', 107), ('she', 105), ('should', 104), ('Thank', 104), ('do', 104), ('Cruz', 102), ('this', 101), ('many', 100), ('We', 98), ('more', 98), ('get', 97), ('your', 96), ('now', 96), ('we', 95), ('no', 94), (

Model building and training

In [ ]:
# Mapping the unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

txt_m_int = np.array([char2idx[c] for c in text])

print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  '$' :   5,
  '%' :   6,
  '&' :   7,
  '(' :   8,
  ')' :   9,
  '+' :  10,
  ',' :  11,
  '-' :  12,
  '.' :  13,
  '/' :  14,
  '0' :  15,
  '1' :  16,
  '2' :  17,
  '3' :  18,
  '4' :  19,
  ...
}


In [ ]:
# The first 13 characters from the text are mapped to integers
print ('{} ---- character mapping to integer ---- > {}'.format(repr(text[:13]), txt_m_int[:13]))

'Busy day plan' ---- characters mapped to int ---- > [30 76 74 80  1 59 56 80  1 71 67 56 69]


In [ ]:
# maximum sentence length
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# training examples/targets
characters= tf.data.Dataset.from_tensor_slices(txt_m_int)

In [ ]:
for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

B
u
s
y
 


In [ ]:
# Converting the individual characters into sequences
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'Busy day planned in New York. Will soon be making some very important decisions on the people who wil'
'l be running our government!\nLove the fact that the small groups of protesters last night have passio'
'n for our great country. We will all come together and be proud!\n"Just had a very open and successful'
' presidential election. Now professional protesters, incited by the media, are protesting. Very unfai'
'r!"\n"A fantastic day in D.C. Met with President Obama for first time. Really good meeting, great chem'


In [ ]:
def split_input_target(letters):
    input_text = letters[:-1]
    target_text = letters[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Busy day planned in New York. Will soon be making some very important decisions on the people who wi'
Target data: 'usy day planned in New York. Will soon be making some very important decisions on the people who wil'


In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected result: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 30 ('B')
  expected output: 76 ('u')
Step    1
  input: 76 ('u')
  expected output: 74 ('s')
Step    2
  input: 74 ('s')
  expected output: 80 ('y')
Step    3
  input: 80 ('y')
  expected output: 1 (' ')
Step    4
  input: 1 (' ')
  expected output: 59 ('d')


In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [ ]:
vocabulary_size= len(vocab)
embedding_dim = 256
num_rnn_units= 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  
  return model

In [ ]:
model = build_model(
  vocabulary_size= len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print("(batch size, seq length, vocabulary_size)")
  print(example_batch_predictions.shape)

(batch size, seq length, vocabulary_size)
(64, 100, 85)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           21760     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 85)            87125     
Total params: 4,047,189
Trainable params: 4,047,189
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

sampled_indices

array([75, 45, 50, 31, 53, 35,  9, 14, 12, 64, 48, 67, 83, 38, 72, 55, 10,
        0, 67, 47, 38, 69, 30, 23, 35, 17, 17, 54,  1, 63, 14, 36, 81,  4,
        1, 15, 16, 33, 14, 49, 14, 38, 36, 18, 81, 66, 42, 26, 13,  2, 82,
       64, 69, 36, 84, 82, 26, 75,  4, 48, 31, 53, 82, 63, 35, 80, 24, 25,
       72, 64, 35, 69,  9, 13, 38, 24, 49, 15, 52,  3, 20, 73, 52,  0, 18,
       46, 62, 28, 37, 52, 64,  5, 13, 59, 50,  7, 28,  6, 58, 54])

In [ ]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Character Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'hen our police are so appreciated!\n"I beat Hillary in the new @FoxNews Poll head to head.  SHE HAS N'

Next Char Predictions: 
 'tQVCYG)/-iTl\u06ddJq_+\nlSJnB8G22Z h/Hz# 01E/U/JH3zkN=.!̱inH۪̱=t#TCY̱hGy9:qiGn).J9U0X"5rX\n3Rg@IXi$.dV&@%cZ'


In [ ]:
# Training 
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("The Prediction Shape=: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("The Scalar Loss value:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 85)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.442887


In [ ]:
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [ ]:
!rm -rf ./training_checkpoints
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS=72

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/72
45/45 [==============================] - 4s 56ms/step - loss: 4.0874 - accuracy: 0.1259
Epoch 2/72
45/45 [==============================] - 3s 55ms/step - loss: 2.7849 - accuracy: 0.2674
Epoch 3/72
45/45 [==============================] - 3s 55ms/step - loss: 2.4891 - accuracy: 0.3123
Epoch 4/72
45/45 [==============================] - 3s 55ms/step - loss: 2.3373 - accuracy: 0.3476
Epoch 5/72
45/45 [==============================] - 3s 56ms/step - loss: 2.1800 - accuracy: 0.3875
Epoch 6/72
45/45 [==============================] - 3s 56ms/step - loss: 2.0223 - accuracy: 0.4323
Epoch 7/72
45/45 [==============================] - 3s 56ms/step - loss: 1.8755 - accuracy: 0.4760
Epoch 8/72
45/45 [==============================] - 3s 56ms/step - loss: 1.7406 - accuracy: 0.5134
Epoch 9/72
45/45 [==============================] - 3s 56ms/step - loss: 1.6194 - accuracy: 0.5458
Epoch 10/72
45/45 [==============================] - 3s 56ms/step - loss: 1.5183 - accuracy: 0.5748
Epoch 11/

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_72'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            21760     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 85)             87125     
Total params: 4,047,189
Trainable params: 4,047,189
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def generate_text(model, start_string):
  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  temperature = 1.0

  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2char[predicted_id])
  return (start_string + ''.join(text_generated))

print(generate_text(model, start_string=u"M"))

MERICA GREAT AGAIN"
"Jeb Bush and Tex Crooked Hillary help, has become a laughingstock rag!"
"Crazy debate, they would allow themselves to be used in a landslide!"
Im going to do what @MittRomney was a disaster. THINK!
Will be interviewed on @oreillyfactor at 8:00 P.M. Enjoy!
"ObamaC, REPEAL AND REPLACE OBAMACARE!
Why has nobody asked Kaine about the 735 delegates - great in states!"
I am in Colorado - big day planning an experts the opening of Trump National Doral on produccional guara). great. The people get it!
Did Crooked Hillary help. Bernie Sanders is lying when her husband was the WORST with everyone in Lafalielake - Russia reprised where Presidential Forum, buries against me at 744 A.M. New Derly durian for me- the truly great competitiors. I۪ve set a change instead of the same.
"Id like people like @ThicWATALEAN: OReilly calls"
"Just landed in New York - a one winner day back an other media solurican for him. Typ funded by  @seanhannity tonight. The great work!"
"Will be inter

In [ ]:
#Advanced
model = build_model(
  vocabulary_size= len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

optimizer = tf.keras.optimizers.Adam()

In [ ]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [ ]:
EPOCHS = 72

for epoch in range(EPOCHS):
  start = time.time()
  hidden = model.reset_states()
  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.443061351776123
Epoch 1 Loss 2.9914
Time taken for 1 epoch 3.7615113258361816 sec

Epoch 2 Batch 0 Loss 2.991408109664917
Epoch 2 Loss 2.5833
Time taken for 1 epoch 2.750640630722046 sec

Epoch 3 Batch 0 Loss 2.5269546508789062
Epoch 3 Loss 2.4054
Time taken for 1 epoch 2.7644169330596924 sec

Epoch 4 Batch 0 Loss 2.411395788192749
Epoch 4 Loss 2.2519
Time taken for 1 epoch 2.7792954444885254 sec

Epoch 5 Batch 0 Loss 2.2273950576782227
Epoch 5 Loss 2.0970
Time taken for 1 epoch 2.807436227798462 sec

Epoch 6 Batch 0 Loss 2.0754811763763428
Epoch 6 Loss 1.9573
Time taken for 1 epoch 2.7839934825897217 sec

Epoch 7 Batch 0 Loss 1.9035476446151733
Epoch 7 Loss 1.8277
Time taken for 1 epoch 2.8010363578796387 sec

Epoch 8 Batch 0 Loss 1.781224250793457
Epoch 8 Loss 1.6841
Time taken for 1 epoch 2.8540990352630615 sec

Epoch 9 Batch 0 Loss 1.6730732917785645
Epoch 9 Loss 1.5622
Time taken for 1 epoch 2.846130132675171 sec

Epoch 10 Batch 0 Loss 1.5662789344787598
Epo

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_71'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            21760     
_________________________________________________________________
gru_3 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_3 (Dense)              (1, None, 85)             87125     
Total params: 4,047,189
Trainable params: 4,047,189
Non-trainable params: 0
_________________________________________________________________


In [74]:
print(generate_text(model, start_string=u"I"))

IN!"
Will be interviewed by Chris Wallace on Fox tomorrow morning. Turning them she has done nothing!
I dont watch or Bernie (Has thr trade deals of buss!"
"I am in Las Vegas, at the best hoteledia digests these numbers - won, by President Pe̱a Nieton one of the best stories ISIS and our ord Clinton does not.
"Hillary can never win over Bernie supporters. Her foreign wqually in the Morning Convenut to have the meeting was stap be on the lookout from which I must be thrilled, then Palinton campaign."
The documentary of me that @KarlRove book is terrible (and boring). Save!
"Wow, Rosanne Brewer Lane, the most prominently depicted woman in the failing @nytimes story yesterday,is on @foxandfriends at 7:30 A.M. Enjoy!
I will be live-tweeting the V.P. pick are the people who have no idea what my strategy on ISIS is, and neither does ISIS (a great Morning Joe at 6:10. Fantastic toposs, many others, said I was on @60Minutes four weeks ago with Putinnever said I was in Green Row an @JuffKerinaR